In [1]:
import numpy as np
import tensorflow as tf
import kerasncp as kncp
from kerasncp.tf import LTCCell
#from kerasncp.ltc_cell import LTCCell
from kerasncp.wirings import NCP
from keras.utils import plot_model
from keras.models import Sequential

In [2]:
from config import GlobalConfig
from data import CARLA_Data
config = GlobalConfig(setting="02_05_withheld")

Skip Town02 and Town05


In [ ]:
train = CARLA_Data(root=config.train_data, config=config).data_generator()

In [3]:

train_set = CARLA_Data(root=config.train_data, config=config).create_dataset()
val_set = CARLA_Data(root=config.val_data, config=config).create_dataset()



100%|██████████| 21/21 [00:02<00:00,  8.07it/s]


In [4]:
x_train = train_set.map(lambda data: data["rgb"])
y_train = train_set.map(lambda data: data["measurements"])

x_val = val_set.map(lambda data: data["rgb"])
y_val = val_set.map(lambda data: data["measurements"])


In [ ]:

for rgb in x_train.take(100):
    print("Input:", rgb.shape)

In [ ]:

for x, y in zip(x_val.take(1), y_val.take(1)):
    print("Input shape:", x.shape)
    print("Output shape:", y.shape)

In [11]:
from keras.layers import (
    Lambda,
    Conv2D,
    Dropout,
    Dense,
    Flatten,
    Reshape,
    RNN,
    Input,
    Add,
    MaxPool2D,
    InputLayer,
    LSTM,
    TimeDistributed,
    Activation
)
from ncps import wirings
import seaborn as sns
import matplotlib.pyplot as plt

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

wiring and model based on Convolutions

In [6]:

"""
Define the Keras sequential CNN-NCP hybrid architecture.

A CNN feature extractor is stacked with an NCP RNN temporal modelling structure.

:return: the CNN-NCP model
"""

# Set the NCP wiring
wiring = NCP(
    inter_neurons=12,  # Number of inter neurons
    command_neurons=8,  # Number of command neurons
    motor_neurons=1,  # Number of motor neurons
    sensory_fanout=4,  # How many outgoing synapses has each sensory neuron
    inter_fanout=4,  # How many outgoing synapses has each inter neuron
    recurrent_command_synapses=4,  # How many recurrent synapses are in the command neuron layer
    motor_fanin=6,  # How many incoming synapses has each motor neuron
)



In [12]:
# Create the the NCP cell based on the LTC neuron.
ncp_cell = LTCCell(wiring)

# Build the sequential hybrid model
model = Sequential(
    [
        Conv2D(24, 5, 2, padding='same', activation="relu", input_shape=(160, 704, 3)),
        Conv2D(36, 5, 2, activation="relu"),
        Conv2D(48, 5, 2, activation="relu"),
        Conv2D(64, 3, activation="relu"),
        Conv2D(64, 3, activation="relu"),
        Dropout(0.5),
        Flatten(),
        Dense(100, activation="relu"),
        Reshape((1, -1)),
        RNN(ncp_cell, unroll=True),
    ]
)

model.compile(
    optimizer=Adam(0.01), loss=MeanSquaredError() , metrics=["mean_squared_error"]
)

#model.summary()
# plot_model(model, show_shapes=True)
# return model

In [8]:
model.evaluate_ge(x_val, y_val)

AttributeError: 'Sequential' object has no attribute 'evaluate_ge'

In [ ]:
hist = model.fit_generator(train_set, epochs=3, validation_data=val_set)

In [ ]:
sns.set_style("white")
plt.figure(figsize=(30, 25))
legend_handles = ncp_cell.draw_graph(layout='shell',neuron_colors={"command": "tab:cyan"})
plt.legend(handles=legend_handles, loc="upper center", bbox_to_anchor=(1, 1))
sns.despine(left=True, bottom=True)
plt.tight_layout()
plt.show()

In [ ]:
ncp_wiring = kncp.wirings.NCP(
    inter_neurons=20,  # Number of inter neurons
    command_neurons=10,  # Number of command neurons
    motor_neurons=5,  # Number of motor neurons
    sensory_fanout=4,  # How many outgoing synapses has each sensory neuron
    inter_fanout=5,  # How many outgoing synapses has each inter neuron
    recurrent_command_synapses=6,  # Now many recurrent synapses are in the
    # command neuron layer
    motor_fanin=4,  # How many incoming synapses has each motor neuron
)
ncp_cell_time = LTCCell(
    ncp_wiring,
    initialization_ranges={
        # Overwrite some of the initialization ranges
        "w": (0.2, 2.0),
    },
)

In [ ]:
time_d_model = Sequential(
    [
        InputLayer(input_shape=(None, 160, 704, 3)),
        TimeDistributed(
            Conv2D(32, (5, 5), activation="relu")
        ),
        TimeDistributed(MaxPool2D()),
        TimeDistributed(
            Conv2D(64, (5, 5), activation="relu")
        ),
        TimeDistributed(MaxPool2D()),
        TimeDistributed(Flatten()),
        TimeDistributed(Dense(32, activation="relu")),
        RNN(ncp_cell_time, return_sequences=True),
        TimeDistributed(Activation("softmax")),
    ]
)
time_d_model.compile(
    optimizer=Adam(0.01),
    loss='sparse_categorical_crossentropy',
)
time_d_model.summary()

In [ ]:
sns.set_style("white")
plt.figure(figsize=(10,10))
legend_handles = ncp_cell_time.draw_graph(layout='spiral',neuron_colors={"command": "tab:cyan"})
plt.legend(handles=legend_handles, loc="upper center", bbox_to_anchor=(1, 1))
sns.despine(left=True, bottom=True)
plt.tight_layout()
plt.show()